In [1]:
%load_ext autoreload
%autoreload 2

In [5]:
# !pip install scanpy

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 67.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.4/124.4 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 84.5 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 82.2 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 61.4 MB/s eta 0:00:0000:010:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.9/233.9 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 MB 56.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 8.5 MB/s eta 0:00:00
  Created wheel for session-info: filename=session_info

# Blood cell data

In [1]:
import scanpy

cell_types = [
    "cd8_cytotoxic_t",
    "cd8_cd45ra_naive_cytotoxic_t",
    "cd14_monocytes",
    "cd19_b",
    "cd34",
    "cd4_cd25_regulatory_t",
    "cd4_cd45ra_cd25_naive_t",
    "cd4_cd45ro_memory_t",
    "cd4_helper_t",
    "cd56_natural_killer",
]

data = {}
for cell_type in cell_types:
    data[cell_type] = scanpy.read_10x_mtx(f"/teamspace/studios/this_studio/embedders/data/blood_cell_scrna/{cell_type}")

In [2]:
# Restrict to landmark genes
import pandas as pd

landmark_genes_table = pd.read_table(
    "/teamspace/studios/this_studio/embedders/data/landmark_genes/GSE92742_Broad_LINCS_gene_info_delta_landmark.txt"
)
landmark_genes = landmark_genes_table["pr_gene_symbol"]

In [3]:
# This seems about right, based on what the Tabaghi paper claimed - we get 967, they get 965

for k, v in data.items():
    print(k, len(set(v.var.index) & set(landmark_genes)), sep="\t")

cd8_cytotoxic_t	967
cd8_cd45ra_naive_cytotoxic_t	967
cd14_monocytes	967
cd19_b	967
cd34	967
cd4_cd25_regulatory_t	967
cd4_cd45ra_cd25_naive_t	967
cd4_cd45ro_memory_t	967
cd4_helper_t	967
cd56_natural_killer	967


In [4]:
# Restrict to landmark genes and annotate cell type

data_filtered = {k: v[:, v.var.index.isin(landmark_genes)] for k, v in data.items()}
for k, v in data_filtered.items():
    v.obs["cell_type"] = k

# Finally, merge
adata = scanpy.AnnData.concatenate(*data_filtered.values(), batch_key="cell_type")
adata

/tmp/ipykernel_24888/310026843.py:5: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  v.obs["cell_type"] = k
/tmp/ipykernel_24888/310026843.py:5: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  v.obs["cell_type"] = k
/tmp/ipykernel_24888/310026843.py:5: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  v.obs["cell_type"] = k
/tmp/ipykernel_24888/310026843.py:5: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  v.obs["cell_type"] = k
/tmp/ipykernel_24888/310026843.py:5: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  v.obs["cell_type"] = k
/tmp/ipykernel_24888/310026843.py:5: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  v.obs["cell_type"] = k
/tmp/ipykernel_24888/3

AnnData object with n_obs × n_vars = 94655 × 967
    obs: 'cell_type'
    var: 'gene_ids'

In [7]:
# Great, that's the correct shape! Let's save it and skip all of this stuff next time

adata.write("/teamspace/studios/this_studio/embedders/data/blood_cell_scrna/adata.h5ad")

# Lymphoma

In [2]:
import scanpy

cell_types = ["lymphoma_cells", "healthy_cells"]

data = {}
# These are gzipped inside
for cell_type in cell_types:
    data[cell_type] = scanpy.read_10x_mtx(
        f"/teamspace/studios/this_studio/embedders/data/blood_cell_scrna/{cell_type}", cache_compression="gzip")

In [3]:
for k, v in data.items():
    v.obs["cell_type"] = k

# Finally, merge
adata = scanpy.AnnData.concatenate(*data.values(), batch_key="cell_type")
adata

# This is the correct number of points for the paper

/tmp/ipykernel_24297/1072584601.py:5: FutureWarning: Use anndata.concat instead of AnnData.concatenate, AnnData.concatenate is deprecated and will be removed in the future. See the tutorial for concat at: https://anndata.readthedocs.io/en/latest/concatenation.html
  adata = scanpy.AnnData.concatenate(*data.values(), batch_key="cell_type")


AnnData object with n_obs × n_vars = 13410 × 1056
    obs: 'cell_type'
    var: 'gene_ids', 'feature_types'

In [4]:
adata.write("/teamspace/studios/this_studio/embedders/data/blood_cell_scrna/adata_lymphoma.h5ad")